# Combination of line plot and heatmap with date axis

Water temperature within a pond was measured for 5 months. Continous wavelet transform enables the identification of frequencies that were prominent within the data. High correlation coefficients indicate high frequency responses. The correlation coefficients were obtained using the `Signal.ContinuousWavelet` module from `FSharp.Stats`.

In [ ]:
// Referencing Plotly.NET.Interactive extension
#r "nuget: Plotly.NET.Interactive, 3.0.2"

open System.IO
open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.LayoutObjects

Installed Packages Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
let waveletData = 
    File.ReadAllLines(__SOURCE_DIRECTORY__ + @"\data\waveletData.txt")
    |> Array.map (fun x -> x.Split '\t')

let waveletPeriods = waveletData.[0].[2..]

let waveletDates,waveletTemp,waveletCoeffs = 
    waveletData
    |> Array.tail
    |> Array.map (fun x -> 
        let date = x.[0]
        let temp = float x.[1]
        let coeff = Array.map float x.[2..] 
        date,temp,coeff
        )
    |> Array.unzip3

In [ ]:
// line chart of raw temperature measurements
let rawChart = 
    Chart.Line (
        x = waveletDates,
        y = waveletTemp,
        LineColor = Color.fromHex "#636efa",
        Name = "temp"
        )
    // sets the id for the axis regarding the line chart
    |> Chart.withAxisAnchor(X=2)
    |> Chart.withAxisAnchor(Y=2) 

// heatmap of continuous wavelet coefficients
let heatmap = 
    let coefficients = waveletCoeffs |> Array.transpose
    Chart.Heatmap(
        coefficients,
        ColorScale = StyleParam.Colorscale.Portland,
        colNames = waveletDates,
        // frequencies that were analysed in wavelet analysis
        rowNames = waveletPeriods,
        // no color scale should be added
        ShowScale = false
        )
    |> Chart.withAxisAnchor(X=1)
    |> Chart.withAxisAnchor(Y=1)

//combine the charts and add additional styling
let waveletChart =
    Chart.combine([heatmap;rawChart])
    //|> Chart.withTemplate(ChartTemplates.lightMirrored)
    // sets bottom x axis for temperature line chart
    |> Chart.withXAxisStyle("Date", Side=StyleParam.Side.Bottom, Id=SubPlotId.XAxis 2, ShowGrid=false)
    // sets top x axis for heatmap
    |> Chart.withXAxisStyle("", Side=StyleParam.Side.Top, ShowLine=false, ShowGrid=false, Id=SubPlotId.XAxis 1, Overlaying=LinearAxisId.X 2)
    // sets left y axis for temperature line chart
    |> Chart.withYAxisStyle("Temperature (°C)", MinMax=(-25.,30.), Side=StyleParam.Side.Left, Id=SubPlotId.YAxis 2)
    // sets right y axis for heatmap
    |> Chart.withYAxisStyle(Title=Title.init("Frequency",X=0.1,Y=0.1), MinMax=(25.,-40.), ShowLine=false, ShowGrid=false, Side=StyleParam.Side.Right, Id=SubPlotId.YAxis 1, Overlaying=StyleParam.LinearAxisId.Y 2)
    |> Chart.withSize(1000.,985.)
    // because just one line trace is present, by default no legend is shown
    |> Chart.withLegend true
    // alters the "Download plot" button to export the plot as SVG 
    |> Chart.withConfig(Config.init(ToImageButtonOptions = ConfigObjects.ToImageButtonOptions.init(Format=ImageFormat.SVG)))
    //|> Chart.withTitle(Title.init("Wavelet transformed temperature data",X=0.5))

waveletChart


<!-- Plotly chart will be drawn inside this DIV -->